# Demonstrating the TableOne package for Python

The `tableone` package is used to create Table 1 for research papers. It was inspired by the R package of the same name.

## Installation

The distribution is hosted on PyPI and directly installable via pip without needing to clone or download this repository. To install the package from PyPI, run the following command in your terminal:

``pip install tableone``

## Example of Table 1 without stratification

In [1]:
# import libraries
from tableone import TableOne
import pandas as pd

In [2]:
# load sample data into a pandas dataframe
url="https://raw.githubusercontent.com/tompollard/data/master/primary-biliary-cirrhosis/pbc.csv"
data=pd.read_csv(url, index_col='id')

In [3]:
data.head()

,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
id,,,,,,,,,,,,,,,,,,,
1,400,2,1.0,58.765229,f,1.0,1.0,1.0,1.0,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
2,4500,0,1.0,56.446270,f,0.0,1.0,1.0,0.0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
3,1012,2,1.0,70.072553,m,0.0,0.0,0.0,0.5,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
4,1925,2,1.0,54.740589,f,0.0,1.0,1.0,0.5,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
5,1504,1,2.0,38.105407,f,0.0,1.0,1.0,0.0,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


In [4]:
# list of columns containing continuous variables
convars = ['time','age','bili','chol','albumin','copper',
           'alk.phos','ast','trig','platelet','protime']

# list of columns containing categorical variables
catvars = ['status', 'ascites', 'hepato', 'spiders', 'edema', 
           'stage', 'sex']

# optionally, a list of non-normal variables
nonnormal = ['bili']

In [5]:
# create an instance of TableOne with the input arguments
# firstly, with no stratifying variable
overall_table = TableOne(data, convars, catvars, nonnormal=nonnormal)

In [6]:
# type the name of the instance in an interpreter to print the table to screen
overall_table

Overall
                       overall
---------------------  -----------------
n                      418
time (mean (std))      1917.78 (1104.67)
age (mean (std))       50.74 (10.45)
bili (median [IQR])    1.40 [0.80,3.40]
chol (mean (std))      369.51 (231.94)
albumin (mean (std))   3.50 (0.42)
copper (mean (std))    97.65 (85.61)
alk.phos (mean (std))  1982.66 (2140.39)
ast (mean (std))       122.56 (56.70)
trig (mean (std))      124.70 (65.15)
platelet (mean (std))  257.02 (98.33)
protime (mean (std))   10.73 (1.02)
status (n (%))
0                      232 (55.50)
1                      25 (5.98)
2                      161 (38.52)
ascites (n (%))
0.0                    288 (92.31)
1.0                    24 (7.69)
hepato (n (%))
0.0                    152 (48.72)
1.0                    160 (51.28)
spiders (n (%))
0.0                    222 (71.15)
1.0                    90 (28.85)
edema (n (%))
0.0                    354 (84.69)
0.5                    44 (10.53)
1.0               

## Example of Table 1 with stratification

In [7]:
# optionally, a categorical variable for stratification
strat = 'trt'

In [8]:
# create an instance of TableOne with the input arguments
strat_table = TableOne(data, convars, catvars, strat, nonnormal)

In [9]:
strat_table

Stratified by trt
                       1.0                2.0
---------------------  -----------------  -----------------
n                      158                154
time (mean (std))      2015.62 (1094.12)  1996.86 (1155.93)
age (mean (std))       51.42 (11.01)      48.58 (9.96)
bili (median [IQR])    1.40 [0.80,3.20]   1.30 [0.72,3.60]
chol (mean (std))      365.01 (209.54)    373.88 (252.48)
albumin (mean (std))   3.52 (0.44)        3.52 (0.40)
copper (mean (std))    97.64 (90.59)      97.65 (80.49)
alk.phos (mean (std))  2021.30 (2183.44)  1943.01 (2101.69)
ast (mean (std))       120.21 (54.52)     124.97 (58.93)
trig (mean (std))      124.14 (71.54)     125.25 (58.52)
platelet (mean (std))  258.75 (100.32)    265.20 (90.73)
protime (mean (std))   10.65 (0.85)       10.80 (1.14)
status (n (%))
0                      83 (52.53)         85 (55.19)
1                      10 (6.33)          9 (5.84)
2                      65 (41.14)         60 (38.96)
ascites (n (%))
0.0           

## Testing

Compute p values by setting the ``pval`` argument to True.

In [10]:
# create strat_table with p values
strat_table = TableOne(data, convars, catvars, strat, nonnormal, pval = True)

In [11]:
strat_table

Stratified by trt
                       1.0                2.0                pval    testname
---------------------  -----------------  -----------------  ------  --------------
n                      158                154
time (mean (std))      2015.62 (1094.12)  1996.86 (1155.93)  0.883   One_way_ANOVA
age (mean (std))       51.42 (11.01)      48.58 (9.96)       0.018   One_way_ANOVA
bili (median [IQR])    1.40 [0.80,3.20]   1.30 [0.72,3.60]   0.842   Kruskal-Wallis
chol (mean (std))      365.01 (209.54)    373.88 (252.48)    0.748   One_way_ANOVA
albumin (mean (std))   3.52 (0.44)        3.52 (0.40)        0.874   One_way_ANOVA
copper (mean (std))    97.64 (90.59)      97.65 (80.49)      0.999   One_way_ANOVA
alk.phos (mean (std))  2021.30 (2183.44)  1943.01 (2101.69)  0.747   One_way_ANOVA
ast (mean (std))       120.21 (54.52)     124.97 (58.93)     0.460   One_way_ANOVA
trig (mean (std))      124.14 (71.54)     125.25 (58.52)     0.886   One_way_ANOVA
platelet (mean (std))  258

## Export the table to file (LaTeX, Markdown, CSV, etc)

Tables can be exported to file in various formats, including:
- LaTeX
- Markdown
- CSV
- HTML

To export the table, call the relevant `to_<format>()` method, with an optional filename.

In [12]:
# Save the overall_table as LaTeX with the default filename ('tableone.tex')
overall_table.to_latex()

In [13]:
# Save the strat_table as markdown with the filename 'strat_table.html'
strat_table.to_html('strat_table.html')